In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import os
from pathlib import Path
import sys

if '..' not in sys.path: sys.path.append('..')

from pydantic import BaseModel
from pydantic_yaml import to_yaml_file, parse_yaml_file_as
from transformers import GPT2Tokenizer

from mllm.exp.cfg import create_mllm_encdec_cfg, create_mllm_ranker_cfg
from mllm.tokenization.chunk_tokenizer import calc_max_inp_size, gen_all_tokens



In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', model_max_length=10000)
tok_dict = gen_all_tokens(tokenizer)
pad_tok, qbeg_tok, qend_tok = tok_dict['pad'].ind, tok_dict['query_begin'].ind, tok_dict['query_end'].ind


In [4]:
model_cfg = create_mllm_ranker_cfg(
    n_vocab=len(tokenizer), inp_len=100, d_word_wec=256,
    n_levels=1, enc_n_layers=1, dec_n_layers=1,
    n_heads=8, d_k=32, d_v=32, d_model=256, d_inner=1024,
    pad_idx=-1, dropout_rate=0.0, enc_with_emb_mat=True,
)
model_cfg

MllmRankerCfg(vocab_encoder=VocabEncoderCfg(n_vocab=50257, d_word_vec=256, d_model=256, pad_idx=-1, inp_len=100, dropout_rate=0.0), encoders=[EncoderCfg(n_layers=1, n_heads=8, d_k=32, d_v=32, d_model=256, d_inner=1024, pad_idx=-1, with_graph_mat=False, inp_len=100, dropout_rate=0.0, with_emb_mat=True)], decoders=[EncoderCfg(n_layers=1, n_heads=8, d_k=32, d_v=32, d_model=256, d_inner=1024, pad_idx=-1, with_graph_mat=False, inp_len=0, dropout_rate=0.0, with_emb_mat=False)])

In [12]:
cfg_dpath = Path(os.path.abspath('.')).parent / 'mllm' / 'config'
cfg_dpath

PosixPath('/home/misha/prog/mllm/mllm/config')

In [11]:
cfg_fpath = cfg_dpath / 'cfg_v001' / 'ranker_model_cfg_01.yaml'
to_yaml_file(cfg_fpath, model_cfg)